In [1]:
!pip install umap
!pip install trimap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=f797896a40ccede6df82ab803c46d5f1cf7b4087b4241e051428ccb00f1485e0
  Stored in directory: /root/.cache/pip/wheels/65/55/85/945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 5.6 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391637 sha256=f4b7d44038a84b5a3775d174d2fb0d1b79295be0a6f0c68be560412c92e75849
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import sys
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code')
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master')

In [5]:
## Importing libraries

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import math
import numpy as np
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models.glyphs import Text
from bokeh.layouts import row
from bokeh.io import export_png
output_notebook()

## Importing local Python files

sys.path.append('../') 
from src import Preprocessing, Distances, Explanation, DR_algorithms, genericMethods
from src.LAPS_tabular import LapsExplainer

## Defining input paths (Update the Data_path attribute accordingly)

Data_path = "/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master/data/"

### Step 1: Load and Pre-process Dataset

#### a. Load Dataset

In [8]:
samples = 100
df = pd.read_csv(Data_path+'Wine_Quality.csv')
df = df.fillna(0)
df = df.sample(samples)
df.reset_index(inplace= True, drop= True)
df.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.24,0.32,1.3,0.037,39.0,123.0,0.99200,3.17,0.42,11.2,8
1,6.4,0.22,0.32,4.9,0.046,50.0,156.0,0.99316,3.38,0.55,11.2,6
2,6.6,0.23,0.30,4.6,0.060,29.0,154.0,0.99142,3.23,0.49,12.2,8
3,6.8,0.35,0.32,2.4,0.048,35.0,103.0,0.99110,3.28,0.46,12.0,8
4,6.8,0.33,0.30,2.1,0.047,35.0,147.0,0.98860,3.24,0.56,13.4,6


#### b. Identify Intrinsic Dimensionality

In [9]:
## Identify intrinsic dimensionality using: the Maximum likelihood intrinsic dimensionality estimator

int_dim = Preprocessing.repeated(Preprocessing.intrinsic_dim_scale_interval, 
                             df.values, 
                             mode='bootstrap', 
                             nb_iter=500, # nb_iter for bootstrapping
                             verbose=1, 
                             k1=10, k2=20)
print ("Intrinsic dimensionality:", int_dim)

100%|██████████| 500/500 [00:05<00:00, 89.74it/s]

Intrinsic dimensionality: 2


#### c. Set Features and Target

In [26]:
model_features, target = Preprocessing.set_features_and_target(df)

y = (df[target].values.reshape(-1, ))
X_df = pd.DataFrame(df, columns=model_features)
data_instance_number = 24
POI = [25, 21]
for i in POI:
  y[i] = -1


Features:  ['fixed.acidity', 'volatile.acidity', 'citric.acid', 'residual.sugar', 'chlorides', 'free.sulfur.dioxide', 'total.sulfur.dioxide', 'density', 'pH', 'sulphates', 'alcohol']
Target:  ['quality']


#### d. Identify Categorical Features

In [27]:
X_transformed, categorical_features, numeric_features, categorical_names = Preprocessing.identify_and_transform_features(df, model_features)

### Step 2: Run Dimensionality Reduction

In [28]:
#ld_embedding = DR_algorithms.run_DR_Algorithm("tSNE", X_transformed)

In [29]:
all_runs = 50
iterations = 300
#data_instance_number = 24

In [ ]:
import my_SNE


oldSNE = my_SNE.My_SNE(X_transformed.T, max_iterations=iterations)
results_orig_SNE = []

ld_embedding_oSNE = []
for _ in range(all_runs):
  x_embed_SNE_old = oldSNE.fit_transform()
  ld_embedding_oSNE = np.array(x_embed_SNE_old[0]).T

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_oSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_orig_SNE.append([components[0], components[1], components[2], divergence])
  #print("Discrepancy in Feature Influence:", components[0])
  #print("Discrepancy in Neighborhood Content:", components[1])
  #print("Discrepancy in Neighborhood Order:", components[2])
  #print("Overall Divergence: ", divergence)
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_oSNE[:,0],
            y=ld_embedding_oSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

In [ ]:
np.save('results_old_50_500', np.array(results_orig_SNE, dtype=np.float32), allow_pickle=True)

In [16]:
print(df.values[:,-1])
print(min(df.values[:,-1]))
print(max(df.values[:,-1]))

[8. 6. 8. 8. 6. 5. 6. 5. 6. 6. 6. 6. 6. 6. 7. 5. 5. 7. 5. 6. 5. 6. 5. 6.
 8. 6. 4. 6. 5. 6. 4. 5. 8. 5. 6. 7. 5. 8. 4. 7. 5. 7. 7. 5. 6. 6. 5. 7.
 6. 6. 5. 6. 5. 6. 7. 4. 5. 5. 5. 6. 6. 5. 5. 4. 5. 5. 7. 6. 6. 4. 5. 6.
 7. 5. 6. 7. 6. 6. 5. 6. 5. 5. 7. 6. 6. 7. 5. 5. 5. 7. 5. 5. 6. 6. 7. 6.
 5. 6. 5. 5.]
4.0
8.0


In [30]:
import new_SNE_torch
nSNE = new_SNE_torch.TorchTSNE(verbose=True, n_iter=iterations)
results_torch_SNE = []

for _ in range(all_runs):
  ld_embedding_torch_SNE = nSNE.fit_transform(X_transformed)

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_torch_SNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_torch_SNE.append([components[0], components[1], components[2], divergence])
indexes = [i for i in range(0,len(y))]
colormap = {4:'Red',5:'Orange',6:'Yellow',7:'Green',8:'Blue',-1:'Purple'}
colors = [colormap[z] for z in y.ravel()]
labels = {4:'4',5:'5',6:'6',7:'7',8:'8',-1:'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_torch_SNE[:,0],
            y=ld_embedding_torch_SNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

using cuda
initializing...
computing p-values...
fitting...
error: 0.933: 100%|██████████| 300/300 [00:00<00:00, 346.08it/s]


point | point_embd
24     |     24
6     |     7
7     |     10
20     |     85
12     |     25
42     |     94
10     |     12
25     |     29
68     |     6
89     |     42
85     |     68
90     |     49
8     |     14
0     |     34
49     |     89
29     |     20
3     |     82
14     |     4
31     |     8
27     |     83


using cuda
initializing...
computing p-values...
fitting...
error: 0.933: 100%|██████████| 300/300 [00:00<00:00, 345.06it/s]


point | point_embd
24     |     24
6     |     7
7     |     10
20     |     85
12     |     25
42     |     94
10     |     12
25     |     29
68     |     6
89     |     42
85     |     68
90     |     49
8     |     14
0     |     34
49     |     89
29     |     20
3     |     82
14     |     4
31     |     8
27     |     83


using cuda
initializing...
computing p-values...
fitting...
error: 0.933: 100%|██████████| 300/300 [00:01<00:00, 256.72it/s]


...

point | point_embd
24     |     24
6     |     7
7     |     10
20     |     85
12     |     25
42     |     94
10     |     12
25     |     29
68     |     6
89     |     42
85     |     68
90     |     49
8     |     14
0     |     34
49     |     89
29     |     20
3     |     82
14     |     4
31     |     8
27     |     83


using cuda
initializing...
computing p-values...
fitting...
error: 0.933: 100%|██████████| 300/300 [00:01<00:00, 271.29it/s]


point | point_embd
24     |     24
6     |     7
7     |     10
20     |     85
12     |     25
42     |     94
10     |     12
25     |     29
68     |     6
89     |     42
85     |     68
90     |     49
8     |     14
0     |     34
49     |     89
29     |     20
3     |     82
14     |     4
31     |     8
27     |     83


using cuda
initializing...
computing p-values...
fitting...
error: 0.933: 100%|██████████| 300/300 [00:00<00:00, 325.05it/s]


point | point_embd
24     |     24
6     |     7
7     |     10
20     |     85
12     |     25
42     |     94
10     |     12
25     |     29
68     |     6
89     |     42
85     |     68
90     |     49
8     |     14
0     |     34
49     |     89
29     |     20
3     |     82
14     |     4
31     |     8
27     |     83


In [31]:
import my_fSNE
weights = [1, 0.9,0.6]
results_fsne_slow = []

fSNE_slow = my_fSNE.my_fSNE(X_transformed, max_iter=iterations, POI = POI, lambdas = weights)

results_torch_fSNE = []
best_score = 1
for _ in range(10):
  ld_embedding_fSNE = fSNE_slow.fit_transform()
  
  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")
  if divergence < best_score:
    best_score = divergence
    np.save('fsne_w_1_09_06.npy', ld_embedding_fSNE, allow_pickle=True)
  results_fsne_slow.append([components[0], components[1], components[2], divergence])
  
ld_embedding_fSNE = np.load('fsne_w_1_09_06.npy', allow_pickle=True)
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")

indexes = [i for i in range(0,len(y))]
colormap = {4:'Red',5:'Orange',6:'Yellow',7:'Green',8:'Blue',-1:'Purple'}
colors = [colormap[z] for z in y.ravel()]
labels = {4:'4',5:'5',6:'6',7:'7',8:'8',-1:'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_fSNE[:,0],
            y=ld_embedding_fSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

Streaming output truncated to the last 5000 lines.
Iteration 1 ...
----Cost: 3.4848073046325796
Iteration 2 ...
----Cost: 3.529767396168794
Iteration 3 ...
----Cost: 3.4738012785992947
Iteration 4 ...
----Cost: 3.74442134952419
Iteration 5 ...
----Cost: 3.860579842476448
Iteration 6 ...
----Cost: 4.026573662041734
Iteration 7 ...
----Cost: 4.095933968699003
Iteration 8 ...
----Cost: 4.282484383488363
Iteration 9 ...
----Cost: 4.124359158754788
Iteration 10 ...
----Cost: 4.01837776708567
Iteration 11 ...
----Cost: 3.885425571979101
Iteration 12 ...
----Cost: 3.8735412070399056
Iteration 13 ...
----Cost: 4.007475163100662
Iteration 14 ...
----Cost: 4.046307158546341
Iteration 15 ...
----Cost: 4.019652177593343
Iteration 16 ...
----Cost: 3.936252419330326
Iteration 17 ...
----Cost: 3.7899775985228334
Iteration 18 ...
----Cost: 3.5070866555798057
Iteration 19 ...
----Cost: 3.7328234700959526
Iteration 20 ...
----Cost: 3.9587463473218465
Iteration 21 ...
----Cost: 3.7947160326418565
Iterati

Best Embedding
POI = [25, 21]
Weights = [1, 0.9, 0.6]
Discrepancy in Feature influence for: fSNE 0.8254563621325438
Discrepancy in Neighborhood Content for: fSNE 0.25
Discrepancy in Feature Order for: fSNE with 0.8
Overall Divergence for: 
fSNE with 0.6251521207108479


In [ ]:
ld_embedding_fSNE = np.load('fsne_w_1_09_06.npy', allow_pickle=True)
data_instance = 24
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance, X_df.values[data_instance], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_fSNE[:,0],
            y=ld_embedding_fSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

point | point_embd
24     |     24
9     |     82
61     |     85
12     |     12
11     |     9
82     |     61
85     |     34
69     |     57
53     |     69
38     |     11
26     |     27
48     |     48
87     |     40
54     |     79
40     |     18
29     |     65
30     |     87
47     |     47
92     |     29
65     |     64


Best Embedding
POI = [24, 86]
Weights = [1, 0.9, 0.6]
Discrepancy in Feature influence for: fSNE 0.7136031822070891
Discrepancy in Neighborhood Content for: fSNE 0.3
Discrepancy in Feature Order for: fSNE with 0.8
Overall Divergence for: 
fSNE with 0.6045343940690296


In [ ]:
'''
import focusedSNE_torch
POI = [data_instance_number, 86]
#POI = []
weights = [2,1,0.75]
fSNE = focusedSNE_torch.TorchTSNE(verbose=True, n_iter=1, points_of_interest = POI, lambdas = weights)

results_torch_fSNE = []
for _ in range(1):
  ld_embedding_fSNE = fSNE.fit_transform(X_transformed)

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_torch_fSNE.append([components[0], components[1], components[2], divergence])
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_fSNE[:,0],
            y=ld_embedding_fSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)
'''

In [33]:
#results_old = np.array(results_orig_SNE, dtype=np.float32)
#results_old = np.load('results_old_50_500.npy', allow_pickle=True)
results_new = np.array(results_torch_SNE, dtype=np.float32)
#results_fnew = np.array(results_torch_fSNE, dtype=np.float32)
results_fslow = np.array(results_fsne_slow, dtype=np.float32)
#mus_old = np.mean(results_old, axis = 0)
mus_new = np.mean(results_new, axis = 0)
#mus_fnew = np.mean(results_fnew, axis = 0)
mus_fnew = np.mean(results_fslow, axis = 0)
print(f'Average of {all_runs} runs.')
print(f'POI = {POI}')
print(f'Discrepancy in Feature influence for: \n, SNE_Torch {mus_new[0]:.4f}, fSNE with weights {weights} {mus_fnew[0]:.4f}')
print(f'Discrepancy in Neighborhood Content for: \n, SNE_Torch {mus_new[1]:.4f}, fSNE with weights {weights} {mus_fnew[1]:.4f}')
print(f'Discrepancy in Feature Order for: \n, SNE_Torch {mus_new[2]:.4f}, fSNE with weights {weights} {mus_fnew[2]:.4f}')
print(f'Overall Divergence for: \n, SNE_Torch {mus_new[3]:.4f}, fSNE with weights {weights} {mus_fnew[3]:.4f}')

Average of 50 runs.
POI = [25, 21]
Discrepancy in Feature influence for: 
, SNE_Torch 0.8263, fSNE with weights [1, 0.9, 0.6] 0.8235
Discrepancy in Neighborhood Content for: 
, SNE_Torch 0.2500, fSNE with weights [1, 0.9, 0.6] 0.2550
Discrepancy in Feature Order for: 
, SNE_Torch 0.9500, fSNE with weights [1, 0.9, 0.6] 0.9000
Overall Divergence for: 
, SNE_Torch 0.6754, fSNE with weights [1, 0.9, 0.6] 0.6595


Average of 50 runs.

POI = [24, 86]

Discrepancy in Feature influence for: 

SNE_Python 0.7142, SNE_Torch 0.7749, fSNE with weights [1, 0.9, 0.6] 0.7169

Discrepancy in Neighborhood Content for: 

SNE_Python 0.4000, SNE_Torch 0.2500, fSNE with weights [1, 0.9, 0.6] 0.2650

Discrepancy in Feature Order for: 

SNE_Python 0.9500, SNE_Torch 0.8500, fSNE with weights [1, 0.9, 0.6] 0.8450

Overall Divergence for: 

SNE_Python 0.6881, SNE_Torch 0.6250, fSNE with weights [1, 0.9, 0.6] 0.6090

Average of 50 runs.

POI = [24, 86]

Discrepancy in Feature influence for: 

SNE_Python 0.7142, SNE_Torch 0.7749, fSNE with weights [1, 1, 1] 0.7183

Discrepancy in Neighborhood Content for: 

SNE_Python 0.4000, SNE_Torch 0.2500, fSNE with weights [1, 1, 1] 0.2500

Discrepancy in Feature Order for: 

SNE_Python 0.9500, SNE_Torch 0.8500, fSNE with weights [1, 1, 1] 0.9000

Overall Divergence for: 

SNE_Python 0.6881, SNE_Torch 0.6250, fSNE with weights [1, 1, 1] 0.6228

In [ ]:
#np.save('fSNE_equalW_interesting.npy', ld_embedding_fSNE,allow_pickle=True)
#ld_embedding_interesting = np.load('fSNE_equalW_interesting.npy', allow_pickle=True)
ld_embedding_interesting = ld_embedding_fSNE
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, 
                                                                                                               ld_embedding_interesting, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")

#newlabels = labels
#for i in POI:
#  newlabels[i] = 2
#  y[i] = 2
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_interesting[:,0],
            y=ld_embedding_interesting[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)
#print(f'Average of {all_runs} runs.')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

point | point_embd
24     |     24
9     |     85
61     |     82
12     |     12
11     |     61
82     |     9
85     |     34
69     |     57
53     |     69
38     |     11
26     |     27
48     |     48
87     |     40
54     |     79
40     |     18
29     |     87
30     |     65
47     |     29
92     |     47
65     |     53


POI = [24, 86]
Weights = [1.5, 1.125, 0.85]
Discrepancy in Feature influence for: fSNE 0.7183304492527608
Discrepancy in Neighborhood Content for: fSNE 0.25
Discrepancy in Feature Order for: fSNE with 0.85
Overall Divergence for: 
fSNE with 0.6061101497509203


In [ ]:
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding[:,0],
            y=ld_embedding[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

### Step 3: Obtain Representative Data-point and Select a Single Point

In [25]:
rep_points = genericMethods.generate_representative_points(20, X_df, X_transformed, y, ld_embedding_torch_SNE)
Rep_order = ["Density Based Outliers", "Points with Highest Density", "Points with Misplaced Neighborhoods", "Points Close to Decision Boundary", "Cluster Centres"]

for index in range(0,len(rep_points)):
    print(Rep_order[index]+" : "+''.join(str(rep_points[index])))

Density Based Outliers : [2, 27, 57, 9, 70, 79, 80, 38, 52, 51, 31, 4, 50, 32, 98, 89, 66, 91, 28, 42]
Points with Highest Density : [2, 27, 57, 9, 70, 79, 80, 38, 52, 51, 31, 4, 50, 32, 98, 89, 66, 91, 28, 42]
Points with Misplaced Neighborhoods : [77, 70, 98, 51, 15, 42, 31, 84, 63, 75, 71, 21, 49, 28, 78, 52, 47, 33, 38, 54]
Points Close to Decision Boundary : [[0, 1, 5, 4, 2, 3], [0, 1, 5, 4, 2, 3], [0, 1, 5, 4, 2, 3], [0, 1, 5, 2, 4, 3], [0, 1, 5, 2, 4, 3], [0, 1, 5, 4, 2, 3], [1, 0, 5, 4, 2, 3], [1, 5, 0, 4, 2, 3], [0, 5, 4, 1, 2, 3], [0, 5, 1, 4, 2, 3], [1, 5, 4, 0, 2, 3], [4, 5, 0, 3, 2, 1], [5, 4, 1, 0, 2, 3], [1, 0, 4, 5, 2, 3], [0, 1, 5, 2, 4, 3], [0, 5, 1, 4, 3, 2], [0, 1, 4, 5, 3, 2], [0, 1, 5, 4, 2, 3], [5, 4, 3, 0, 1, 2], [0, 1, 5, 2, 4, 3]]
Cluster Centres : [25, 21]


In [ ]:
rep_points = genericMethods.generate_representative_points(20, X_df, X_transformed, y, ld_embedding_torch_SNE)
Rep_order = ["Density Based Outliers", "Points with Highest Density", "Points with Misplaced Neighborhoods", "Points Close to Decision Boundary", "Cluster Centres"]

for index in range(0,len(rep_points)):
    print(Rep_order[index]+" : "+''.join(str(rep_points[index])))

Density Based Outliers : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Highest Density : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Misplaced Neighborhoods : [44, 27, 48, 16, 89, 24, 55, 40, 2, 62, 84, 13, 83, 72, 74, 23, 29, 46, 25, 28]
Points Close to Decision Boundary : [88, 33, 39, 62, 18, 97, 17, 29, 16, 47, 46, 43, 2, 21, 48, 6, 75, 69, 8, 44]
Cluster Centres : [24, 86]


In [ ]:
rep_points = genericMethods.generate_representative_points(20, X_df, X_transformed, y, ld_embedding_fSNE)
Rep_order = ["Density Based Outliers", "Points with Highest Density", "Points with Misplaced Neighborhoods", "Points Close to Decision Boundary", "Cluster Centres"]

for index in range(0,len(rep_points)):
    print(Rep_order[index]+" : "+''.join(str(rep_points[index])))

Density Based Outliers : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Highest Density : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Misplaced Neighborhoods : [44, 48, 16, 27, 89, 24, 55, 62, 2, 40, 84, 6, 25, 36, 72, 74, 29, 28, 7, 13]
Points Close to Decision Boundary : [88, 33, 39, 62, 18, 97, 17, 29, 16, 47, 46, 43, 2, 21, 48, 6, 75, 69, 8, 44]
Cluster Centres : [24, 86]


In [ ]:
## user selects an instance

data_instance_number = 12

### Step 4: Initiate and Execute LAPS Explainer

#### a. Initiate Explainer

In [ ]:
explainer = LapsExplainer(X_df.values,
                feature_names=model_features,
                class_names=target,
                categorical_features=categorical_features,
                categorical_names=categorical_names,
                discretize_continuous=False,
                discretizer='quartile',
                random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(
                                                                                X_df.values,
                                                                                X_transformed,
                                                                                ld_embedding,
                                                                                data_instance_number,
                                                                                X_df.values[data_instance_number],
                                                                                nbrs=20,
                                                                                num_features=5)

#### b. Explain the Point in Original Data and in Embedding

In [ ]:
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], 
                                                                                                              neighbors, 
                                                                                                              oversampled_data, 
                                                                                                              model_features, 
                                                                                                              categorical_features, 
                                                                                                              numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], 
                                                                                                                                       neighbors_embd, 
                                                                                                                                       oversampled_data_embd, 
                                                                                                                                       model_features, 
                                                                                                                                       categorical_features, 
                                                                                                                                       numeric_features)

#### c. Plot Feature Contribution in Original Data and in Embedding

In [ ]:
bar_color = []
y_offset_val = []
label = []
for item in feature_dict.values():
    label.append("{0:.2f}".format(item))
    if item<0:
        bar_color.append('#e34a33')
        y_offset_val.append(item+15)
    else:
        bar_color.append('#2ca25f')
        y_offset_val.append(item)
        
        
source = ColumnDataSource(
        data=dict(
            x_val = list(feature_dict.keys()),
            y_val = list(feature_dict.values()),
            y_offset = y_offset_val,
            labels =  label,
            color = bar_color
        )
    )

p = figure(x_range=list(feature_dict.keys()), plot_height=380, plot_width=980, toolbar_location=None, tools="")

glyph = Text(x="x_val", y="y_val", text='labels', x_offset=-15, y_offset='y_offset', text_font_size="9pt", text_font_style = 'bold', text_color="black")


p.vbar(x="x_val", top="y_val", color="color", width=0.7, source=source)
p.xaxis.major_label_orientation = math.pi/7
p.xaxis.major_label_text_font_size = '9pt'
p.yaxis.major_label_text_font_size = '9pt'
p.xaxis.major_label_text_font_style = 'bold'
p.yaxis.major_label_text_font_style = 'bold'
p.min_border_left = 82
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_align = 'center'
p.y_range.start = np.min(list(feature_dict.values()))-0.05
p.y_range.end = np.max(list(feature_dict.values()))+0.01
p.add_glyph(source, glyph)
#export_png(p, filename="Images/Fig4/Feature-Influences-85-Original.png")
show(p)

In [ ]:
bar_color = []
y_offset_val = []
label = []
for item in feature_dict_embd.values():
    label.append("{0:.2f}".format(item))
    if item<0:
        bar_color.append('#e34a33')
        y_offset_val.append(item+15)
    else:
        bar_color.append('#2ca25f')
        y_offset_val.append(item)
        
        
source = ColumnDataSource(
        data=dict(
            x_val = list(feature_dict_embd.keys()),
            y_val = list(feature_dict_embd.values()),
            y_offset = y_offset_val,
            labels =  label,
            color = bar_color
        )
    )

p = figure(x_range=list(feature_dict_embd.keys()), plot_height=380, plot_width=980, toolbar_location=None, tools="")

glyph = Text(x="x_val", y="y_val", text='labels', x_offset=-15, y_offset='y_offset', text_font_size="9pt", text_font_style = 'bold', text_color="black")


p.vbar(x="x_val", top="y_val", color="color", width=0.7, source=source)
p.xaxis.major_label_orientation = math.pi/7
p.xaxis.major_label_text_font_size = '9pt'
p.yaxis.major_label_text_font_size = '9pt'
p.xaxis.major_label_text_font_style = 'bold'
p.yaxis.major_label_text_font_style = 'bold'
p.min_border_left = 80
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_align = 'center'
p.y_range.start = np.min(list(feature_dict.values()))-0.05
p.y_range.end = np.max(list(feature_dict.values()))+0.01
p.add_glyph(source, glyph)
#export_png(p, filename="Images/Fig4/Feature-Influences-85-Embedding.png")
show(p)

#### d. Calculate Local Divergence

In [ ]:
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")

print("Discrepancy in Feature Influence:", components[0])
print("Discrepancy in Neighborhood Content:", components[1])
print("Discrepancy in Neighborhood Order:", components[2])
print("Overall Divergence: ", divergence)

Discrepancy in Feature Influence: 0.7374545801730664

Discrepancy in Neighborhood Content: 0.35

Discrepancy in Neighborhood Order: 0.9

Overall Divergence:  0.6624848600576887